# Analytische Vaardigheden




Importeren van de dependencies


In [174]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import datetime
pd.options.mode.chained_assignment = None

## Het inladen van de dataset
Eerst laden we de dataset in

In [175]:
indexes = pd.read_csv('GSPC.csv')
indexes.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1950-01-03,16.66,16.66,16.66,16.66,16.66,1260000
1,1950-01-04,16.85,16.85,16.85,16.85,16.85,1890000
2,1950-01-05,16.93,16.93,16.93,16.93,16.93,2550000
3,1950-01-06,16.98,16.98,16.98,16.98,16.98,2010000
4,1950-01-09,17.08,17.08,17.08,17.08,17.08,2520000


## Het schoonmaken
Dan gaan we de dataset schoonmaken en alle kolommen droppen die we niet meer gaan gebruiken. Ook maken we van de kolomnamen kleine letters want dat is handig!

In [176]:
indexes['date'] = pd.to_datetime(indexes['Date'])
indexes['open'] = indexes['Open']
columns_to_keep = [
    'date',
    'open'
]
indexes = indexes[columns_to_keep]
indexes.head()

,date,open
0,1950-01-03,16.66
1,1950-01-04,16.85
2,1950-01-05,16.93
3,1950-01-06,16.98
4,1950-01-09,17.08


Nu gaan we de startjaar en eindjaar bepalen

In [177]:
startjaar = 2013
duur = 4
eindjaar = startjaar + duur
eindjaar

2017

Stel de rente in op de lening en het jaarlijks rendement op de staatslening beleggingen

In [178]:
leningrente = 0.0234
bondsrendement = 0.0171

Nu gaan we een nieuwe Dataframe maken waar al onze resultaten in komen, we maken er eerst eentje met alle datums waar we gaan investeren. 

In [179]:
jaren = pd.date_range(str(startjaar) + "-09-01", periods=duur + 1, freq='AS-SEP')
df = pd.DataFrame({'date': jaren, 'schuld met rente 0.00': 0.0, 'schuld met rente {}'.format(leningrente*100): 0.0})
df.head()

,date,schuld met rente 0.00,schuld met rente 2.34
0,2013-09-01,0.0,0.0
1,2014-09-01,0.0,0.0
2,2015-09-01,0.0,0.0
3,2016-09-01,0.0,0.0
4,2017-09-01,0.0,0.0


Nu gaan we de maandelijkse studiefinanciering bepalen, deze is te vinden op de site van DUO.

In [180]:
lening = 486.08
beurs = 396.39
collegegeldkrediet = 173.59
totaal = lening + beurs + collegegeldkrediet
totaal

1056.06

Dat is wat we per maand kunnen krijgen van Ome duo. Per jaar is dat dus:

In [181]:
totaal = totaal * 12
totaal

12672.72

Op 1 september van het eerste jaar hebben we nog geen geld, want dan beginnen we pas, dus dan is de schuld 0

In [182]:
df['schuld met rente 0.00'][0] = 0
df['schuld met rente {}'.format(leningrente*100)][0] = 0
df.head()

,date,schuld met rente 0.00,schuld met rente 2.34
0,2013-09-01,0.0,0.0
1,2014-09-01,0.0,0.0
2,2015-09-01,0.0,0.0
3,2016-09-01,0.0,0.0
4,2017-09-01,0.0,0.0


Doordat er in de dataset niet altijd een 1 september te vinden is, omdat er in het weekend de markten niet open zijn, gaan we kijken wat de eerstvolgende datum is dat er wel een waarde beschikbaar is

In [183]:
def get_day_value(date):
    global df, indexes
    if len(indexes[indexes['date'] != date]) != len(indexes) - 1:
        return get_day_value(date + datetime.timedelta(days=1))
    else:
        return indexes[indexes['date'] == date]

for index, row in df.iterrows():
    df['date'][index] = get_day_value(row['date']).iloc[0]['date']
df.head()
df = pd.merge(df, indexes, on='date')
df.head()

,date,schuld met rente 0.00,schuld met rente 2.34,open
0,2013-09-03,0.0,0.0,1635.949951
1,2014-09-02,0.0,0.0,2004.069946
2,2015-09-01,0.0,0.0,1970.089966
3,2016-09-01,0.0,0.0,2171.330078
4,2017-09-01,0.0,0.0,2474.419922


We voegen nog de kolommen toe voor de aantal aandelen en voor de rest van het geld

In [184]:
df['aandelen'] = 0
df['rest'] = 0
df.head()

,date,schuld met rente 0.00,schuld met rente 2.34,open,aandelen,rest
0,2013-09-03,0.0,0.0,1635.949951,0,0
1,2014-09-02,0.0,0.0,2004.069946,0,0
2,2015-09-01,0.0,0.0,1970.089966,0,0
3,2016-09-01,0.0,0.0,2171.330078,0,0
4,2017-09-01,0.0,0.0,2474.419922,0,0


Nu komt het gedeelte waar we elk jaar berekenen 

In [185]:
for index, row in df.iterrows():
    if index == 0:
        # Jaar 1 totaal = schuld
        df.loc[index, 'schuld met rente 0.00'] = totaal
        df.loc[index, 'schuld met rente {}'.format(leningrente*100)] = totaal
        df.loc[index, 'aandelen'] = np.floor(totaal / row['open'])
        df.loc[index, 'rest'] = totaal - df.loc[index, 'aandelen'] * row['open']
    elif index != len(df) - 1:
        # Neem rente mee in de schuld = totaal * (1 + rente)^aantal jaren
        df.loc[index, 'schuld met rente 0.00'] = df.iloc[index - 1]['schuld met rente 0.00'] + totaal
        df.loc[index, 'schuld met rente {}'.format(leningrente*100)] = round(df.iloc[index - 1]['schuld met rente {}'.format(leningrente*100)] + (totaal * (1 + leningrente)**index),2)
        df.loc[index, 'aandelen'] = df.loc[index - 1, 'aandelen'] + np.floor((totaal + df.loc[index - 1, 'rest']) / row['open'])
        df.loc[index, 'rest'] = (totaal + df.loc[index -1, 'rest']) - np.floor((totaal + df.loc[index - 1, 'rest']) / row['open']) * row['open']
    else:
        # Neem rente mee in eindstand, voeg rente toe op de vorige schuld
        df.loc[index, 'schuld met rente 0.00'] = round(df.loc[index - 1, 'schuld met rente 0.00'])
        df.loc[index, 'schuld met rente {}'.format(leningrente*100)] = round(df.loc[index - 1, 'schuld met rente {}'.format(leningrente*100)] * (1 + leningrente ), 2)
        df.loc[index, 'aandelen'] = df.loc[index - 1, 'aandelen']
        df.loc[index, 'rest'] = df.loc[index - 1, 'rest']
        
df.head()

,date,schuld met rente 0.00,schuld met rente 2.34,open,aandelen,rest
0,2013-09-03,12672.72,12672.72,1635.949951,7.0,1221.070343
1,2014-09-02,25345.44,25641.98,2004.069946,13.0,1869.370667
2,2015-09-01,38018.16,38914.72,1970.089966,20.0,751.460905
3,2016-09-01,50690.88,52498.04,2171.330078,26.0,396.200437
4,2017-09-01,50691.00,53726.49,2474.419922,26.0,396.200437


Eind portofolio waarde SP500:

In [186]:
round(df.loc[len(df) - 1, 'aandelen'] * df.loc[len(df) - 1, 'open'] + df.loc[len(df) - 1, 'rest'], 2)

64731.12

Totaal rendement bij SP500 met en zonder rente

In [187]:
sp500zonderrente = round(df.loc[len(df) - 1, 'aandelen'] * df.loc[len(df) - 1, 'open'] + df.loc[len(df) - 1, 'rest'] - df.loc[len(df) -1, 'schuld met rente 0.00'], 2)
sp500metrente = round(df.loc[len(df) - 1, 'aandelen'] * df.loc[len(df) - 1, 'open'] + df.loc[len(df) - 1, 'rest'] - df.loc[len(df) -1, 'schuld met rente {}'.format(leningrente*100)], 2)
staatsleningenzonderrente = 1
staatsleningenmetrente = 1


# Print de eindstand
print('Zonder rente levert de SP500 {} euro op \nMet een rente van {}% levert de SP500 {} euro op.'.format(sp500zonderrente, leningrente*100, sp500metrente))

Zonder rente levert de SP500 14040.12 euro op 
Met een rente van 2.34% levert de SP500 11004.63 euro op.
